In [2]:
import os
import re
import pandas as pd
import panel as pn
import param
import os
import holoviews as hv

import pandas as pd
import plotly.graph_objects as go

# perspective packages
import pickle
from pathlib import Path
import datetime as dt

In [2]:
preds_df = pd.read_csv('experiments/', index_col=0)
preds_df

,pricing_date,preds,target,forecast_date
0,2023-01-02,1.816086,1.100,2023-01-02
1,2023-01-03,1.799319,0.800,2023-01-02
2,2023-01-04,1.960798,0.350,2023-01-02
3,2023-01-05,2.026101,-0.120,2023-01-02
4,2023-01-06,1.886216,-0.570,2023-01-02
...,...,...,...,...
2505,2023-12-25,2.946715,3.015,2023-12-18
2506,2023-12-26,2.931898,3.055,2023-12-18
2507,2023-12-27,2.702930,3.135,2023-12-18
2508,2023-12-28,2.361506,3.255,2023-12-18


In [3]:
raw_df = pd.read_csv('data_2024-01-04.csv')
raw_df

,pricing_date,ALVARI,ARAMCOSG,ARAMCOTF,ARCENERGY,BBEN,BPSG,BRIGHTOILSG,BUYER1,BUYER2,...,_rgp_03_sz_3,_rgp_04_sz_3,_rgp_05_sz_3,_rgp_06_sz_3,_rgp_07_sz_3,_rgp_08_sz_3,_rgp_09_sz_3,_rgp_10_sz_3,_rgp_11_sz_3,F380 M2/M3
0,2017-01-10,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.50
1,2017-01-11,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.30
2,2017-01-12,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,530.0,0.000000,920.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.70
3,2017-01-13,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,1060.0,0.000000,615.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.85
4,2017-01-16,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,1060.0,0.000000,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,2024-01-29,0.0,0.0,0.0,0.0,0.0,6.666667,0.0,0.0,0.0,...,160.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
1840,2024-01-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
1841,2024-01-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
1842,2024-02-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,80.000000,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN


In [4]:
preds_df.set_index('pricing_date', inplace=True)
preds_df.index = pd.to_datetime(preds_df.index)
preds_df = preds_df.drop(columns='target')
preds_df

,preds,forecast_date
pricing_date,,
2023-01-02,1.816086,2023-01-02
2023-01-03,1.799319,2023-01-02
2023-01-04,1.960798,2023-01-02
2023-01-05,2.026101,2023-01-02
2023-01-06,1.886216,2023-01-02
...,...,...
2023-12-25,2.946715,2023-12-18
2023-12-26,2.931898,2023-12-18
2023-12-27,2.702930,2023-12-18


In [5]:
raw_df.set_index('pricing_date', inplace=True)
raw_df.index = pd.to_datetime(raw_df.index)

In [6]:
pn.config.js_files["jquery"]="https://code.jquery.com/jquery-1.12.4.min.js"
pn.config.js_files["sparkline"]="https://cdnjs.cloudflare.com/ajax/libs/jquery-sparklines/2.1.2/jquery.sparkline.min.js"

pn.extension('tabulator', 'plotly', 'ipywidgets', 'perspective',  css_files=[pn.io.resources.CSS_URLS['font-awesome']], sizing_mode='stretch_width', omms='ipywidgets')

pn.config.js_files["sparkline_to_tabulator"]="https://cdn.jsdelivr.net/gh/MarcSkovMadsen/awesome-panel-assets@master/js/add_spark_lines_to_tabulator.min.js"


In [7]:
def custom_pnl_10(y_pred, y_true):
    nth = len(y_pred)

    thresh = 0.2
    
    try:
        diff = y_pred[-1] - y_pred[0]
    except IndexError:
        diff = y_pred[-1] - y_pred[0]

    direction = 1 if diff > thresh else (-1 if diff < thresh else 0)
    pnl = diff * direction

    return 'pnl', pnl

In [8]:
import numpy as np


def get_preds(preds, date):
    preds_df = preds.copy()
    preds_df = preds_df[preds_df['forecast_date'] == date]

    if 'index' in preds_df.columns:
        idx = np.unique(preds_df['index'], return_index=True)[1]
    else:
        idx = np.unique(preds_df.index, return_index=True)[1]
    return preds_df.iloc[idx]

In [9]:
import param
import numpy as np
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

SELECTED_RANGE = None

forecast_dates = sorted(list(preds_df['forecast_date'].unique()))

class Model(param.Parameterized):
    format_date = lambda x: [int(y) for y in x.split('-')]
    start_date, end_date = dt.datetime(*format_date(forecast_dates[0])), dt.datetime(*format_date(forecast_dates[-1]))
    values = (start_date, end_date)
    period = param.DateRange(default=values, bounds=(start_date, end_date))

def select_available_dates(values):
    available = []

    for option in checkboxes.values:
        opt = datetime(*[int(x) for x in option.split('-')])
        if values[0] <= opt <= values[1]:
            available.append(option)
            
    return available

def out(value):
    checkboxes.value = select_available_dates(value)

model = Model()
date_picker = pn.widgets.DatetimeRangePicker.from_param(model.param.period, enable_time=False)
checkboxes = pn.widgets.CheckBoxGroup(name='Checkbox Group', value=[forecast_dates[0]], options=forecast_dates, inline=False, margin=(20, 10, 10, 10))

select = get_preds(preds_df, forecast_dates[0])
target = raw_df[(raw_df.index >= select.index.min()) & (raw_df.index <= select.index.max())]['F380 M2/M3']
select['target'] = target.values
TARGETS = {}

def create_pnl(df_copy):
    results = {}

    uniq = df_copy.forecast_date.unique()

    for date in uniq:
        d = df_copy[df_copy.forecast_date == date]
        res = custom_pnl_10(d.preds.values, d.target.values)

        if any(TARGETS[date].target.isna().tolist()):
            results[date] = 'unknown'
        else:
            results[date] = res[1]

    return pd.DataFrame(results.values(), index=results.keys(), columns=['pnl'])


@pn.depends(checkboxes.param.value)
def update_pnl_df(sel):
    if SELECTED_RANGE is None:
        return create_pnl(SELECTED_RANGE)
    return create_pnl(SELECTED_RANGE)

@pn.depends(checkboxes.param.value)
def update_plot(selected):
    global SELECTED_RANGE, dataframe
    
    sel = preds_df['forecast_date'].apply(lambda x: x in selected)
    selected_range = preds_df[sel]
    selected_range.reset_index(inplace=True)
    print(selected_range)
    SELECTED_RANGE = selected_range
    
    SELECTED_RANGE['target'] = None

    fig = go.Figure()
    
    for unique_date in selected_range.forecast_date.unique():
        select = get_preds(selected_range, unique_date)
        fig.add_trace(go.Scatter(x=select.pricing_date, y=select.preds, name='Preds:' + unique_date, connectgaps=False))

        # get real target values 
        target = raw_df[(raw_df.index >= select.pricing_date.min()) & (raw_df.index <= select.pricing_date.max())]['F380 M2/M3']
        target.index = pd.to_datetime(target.index)
        target.interpolate(method='linear', inplace=True)
        fig.add_trace(go.Scatter(x=target.index, y=target.values, name='Hist:' + unique_date, connectgaps=False, line=dict(color='red')))

        target = pd.DataFrame({'index': target.index, 'target': target.values})
        target['index'] = pd.to_datetime(target['index'])

        TARGETS[unique_date] = target

    return fig


sidebar = pn.Column(date_picker, pn.bind(out, model.param.period), checkboxes)

widget = pn.WidgetBox(pn.Row(pn.Column(update_plot, width=1000), pn.Column(update_pnl_df, width=200), row_height=800))
main = pn.template.FastGridTemplate(main=widget, sidebar=sidebar, width=1200, main_max_width='100%', row_height=800)
main.show()

     pricing_date     preds forecast_date
0      2023-01-02  1.816086    2023-01-02
1      2023-01-03  1.799319    2023-01-02
2      2023-01-04  1.960798    2023-01-02
3      2023-01-05  2.026101    2023-01-02
4      2023-01-06  1.886216    2023-01-02
...           ...       ...           ...
2505   2023-12-25  2.946715    2023-12-18
2506   2023-12-26  2.931898    2023-12-18
2507   2023-12-27  2.702930    2023-12-18
2508   2023-12-28  2.361506    2023-12-18
2509   2023-12-29  2.115078    2023-12-18

[2510 rows x 3 columns]


C:\Users\dinar\AppData\Local\Temp\ipykernel_21268\3802953853.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dinar\AppData\Local\Temp\ipykernel_21268\3802953853.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dinar\AppData\Local\Temp\ipykernel_21268\3802953853.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dinar\AppData\Local\Temp\ipykernel_21268\3802953853.py:84: SettingWithCopyWarning:


A v

Launching server at http://localhost:60300


In [10]:
raw_df

,ALVARI,ARAMCOSG,ARAMCOTF,ARCENERGY,BBEN,BPSG,BRIGHTOILSG,BUYER1,BUYER2,CAOSG,...,_rgp_03_sz_3,_rgp_04_sz_3,_rgp_05_sz_3,_rgp_06_sz_3,_rgp_07_sz_3,_rgp_08_sz_3,_rgp_09_sz_3,_rgp_10_sz_3,_rgp_11_sz_3,F380 M2/M3
pricing_date,,,,,,,,,,,,,,,,,,,,,
2017-01-10,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.50
2017-01-11,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.30
2017-01-12,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,530.0,0.000000,920.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.70
2017-01-13,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,1060.0,0.000000,615.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.85
2017-01-16,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,1060.0,0.000000,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-29,0.0,0.0,0.0,0.0,0.0,6.666667,0.0,0.0,0.0,0.0,...,160.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
2024-01-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
2024-01-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,133.333333,0.000000,0.0,0.0,0.0,216.0,0.0,0.0,NaN
